In [1]:
import json
import folium
import requests
import time
import pandas as pd

In [2]:
with open('bah_data.json') as json_file:
    city_js = json.load(json_file)

In [136]:
def get_loc(city):
    url = f'https://eu1.locationiq.com/v1/search.php?key=9cd4fb43b10b80&q={city}&format=json'
    resp = requests.get(url)
    js = json.loads(resp.text)
    return (float(js[0]['lat']), float(js[0]['lon']))

In [137]:
for k, v in city_js.items():
    try:
        city_js[k] = {'price':v, 'location': get_loc(k)}
        time.sleep(2)
    except Exception as e:
        print(e)

In [14]:
with open('bah_data.json', 'w+') as f:
    json.dump(city_js, f)

In [3]:
m = folium.Map(location=[32.900908, -97.040335], zoom_start=4)

In [4]:
for city_name, city_props in city_js.items():
    dot = folium.Circle(location=city_props['location'], radius=10000, tooltip=str(city_name) + ' - ' 
                        + str(city_props['price']) + ' $', color = 'black')
    m.add_child(dot)


In [5]:
m

In [6]:
folium.GeoJson(
    'https://raw.githubusercontent.com/shawnbot/topogram/master/data/us-states.geojson',
    name='geojson'
).add_to(m)

In [7]:
geo_json_url = 'https://raw.githubusercontent.com/shawnbot/topogram/master/data/us-states.geojson'

In [8]:
import math
def get_distance(loc1, loc2):
    lat1, lon1 = loc1[0], loc1[1]
    lat2, lon2 = loc2[0], loc2[1]

    R = 6371;
    f1 = math.radians(lat1)
    f2 = math.radians(lat2)
    d_lat = math.radians((lat2-lat1))
    d_lon = math.radians((lon2-lon1))

    a = math.pow(math.sin(d_lat/2),2) + math.cos(f1) * math.cos(f2) * math.pow(math.sin(d_lon/2),2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
    
    return R * c 

In [9]:
city_df=pd.DataFrame()
for item in city_js.items():
    city_df = city_df.append({'CITY': item[0][:-4], 'STATE': item[0][-2:], 'PRICE': item[1]['price']}, ignore_index=True)

In [10]:
state_prices = city_df.groupby('STATE', as_index=False).mean()

In [11]:
m = folium.Map(location=[32.900908, -97.040335], zoom_start=4)

In [12]:
folium.Choropleth(
    geo_data=geo_json_url,
    name='choropleth',
    data=state_prices,
    columns=['STATE', 'PRICE'],
    key_on='feature.properties.postal',
    bins=9,
    fill_color='PuRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average BAH(E05 grade, w/ dependents) rates ($)'
).add_to(m)

folium.LayerControl().add_to(m)

In [13]:
m

In [108]:
m.save('C:/Users/tamas.hegedus/Desktop/BAH_prices.html')